In [1]:
from adding_nop import *
from common_function import *

if __name__ == '__main__':
        
    sample_dir = '../sample/section_move_sample/'
    save_dir = '../sample/perturbated_sample/adding_nop/'
    
    samples = list_files_by_size(sample_dir)
    create_directory(save_dir)
    
    number_of_nop = 15
    
    for sample in samples:
        save_dir = '../sample/perturbated_sample/adding_nop/'
        if '.ipynb' in sample or '.pickle' in sample or '.txt' in sample or '.zip' in sample:
            continue

#         if 'PEview_new.exe' not in sample:
#         if '0c34ed46c75b33e392091d8fb7b4449b2fd78b6a56ae7d89f5e6441c48f10692_new.exe' in sample or 'PEview_new.exe' in sample or 'hello_32_new.exe' in sample:
#         if 'hello_32_new.exe' not in sample:
#             continue

        file_path = sample_dir+sample

        print(file_path)
                 
        new_text, modified_address, caller_callee_dict, checking_target_address = make_new_text(file_path, number_of_nop)

        if new_text is None:
            print(f"[+] Error : failed to make new_text section.") 
        else:
            print("len_new_text: ",len(new_text))
            new_text = modify_headers(file_path, new_text)
            save_dir = modify_section(file_path, new_text, save_dir,number_of_nop)
            modify_tramp(save_dir, modified_address)
            save_dir = modify_rdata(save_dir, modified_address)
            #valid_address_check(file_path, save_dir, caller_callee_dict, checking_target_address, modified_address, str(number_of_nop))
            print("Done!!",sample)

Directory '../sample/perturbated_sample/adding_nop/' created successfully.
../sample/section_move_sample/PEview_new.exe
32


10088it [08:29, 19.81it/s]

len_new_text:  145096
modified_section_return save_dir :  ../sample/perturbated_sample/adding_nop/PEview_new_adding_15.exe
Modified PE file saved as ../sample/perturbated_sample/adding_nop/PEview_new_adding_15.exe
{}
Done!! PEview_new.exe
